In [69]:
#just importing the important libraries, which will come handy along the process
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [70]:
data=pd.read_excel("Cryotherapy.xlsx")

In [72]:
feature=data.drop('Result_of_Treatment',axis=1).copy()
feature=np.array(feature),[90,6]
print(feature[1].shape)
target=data['Result_of_Treatment']

(6,)


In [3]:
data.head(10)

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
1,1,29,7.00,5,1,96,1
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0
5,1,41,8.00,2,2,20,1
6,1,36,11.00,2,1,8,0
7,1,59,3.50,3,3,20,0
8,1,20,4.50,12,1,6,1
9,2,34,11.25,3,3,150,0


In [4]:
data.isna().sum()

sex                    0
age                    0
Time                   0
Number_of_Warts        0
Type                   0
Area                   0
Result_of_Treatment    0
dtype: int64

In [5]:
data.describe()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,1.477778,28.600000,7.666667,5.511111,1.700000,85.833333,0.533333
std,0.502304,13.360852,3.406661,3.567155,0.905042,131.733153,0.501683
min,1.000000,15.000000,0.250000,1.000000,1.000000,4.000000,0.000000
25%,1.000000,18.000000,4.562500,2.000000,1.000000,20.000000,0.000000
50%,1.000000,25.500000,8.500000,5.000000,1.000000,70.000000,1.000000
75%,2.000000,35.000000,10.687500,8.000000,3.000000,100.000000,1.000000
max,2.000000,67.000000,12.000000,12.000000,3.000000,750.000000,1.000000


In [18]:
#just seperate the positive and negative data , so that, it is easy to find mean , variance and standard deviation 
pos_data=data[data['Result_of_Treatment']==1]
pos_feature=pos_data.drop('Result_of_Treatment', axis=1)
neg_data=data[data['Result_of_Treatment']==0]
neg_feature=pos_data.drop('Result_of_Treatment', axis=1)

In [19]:
pos_feature.head()

,sex,age,Time,Number_of_Warts,Type,Area
1,1,29,7.0,5,1,96
5,1,41,8.0,2,2,20
8,1,20,4.5,12,1,6
11,2,15,6.0,2,1,30
12,2,15,2.0,3,1,4


In [20]:
neg_data.head()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0
6,1,36,11.00,2,1,8,0


In [21]:
pos_mean=np.mean(pos_feature, axis=0)
neg_mean=np.mean(neg_feature, axis=0)

In [22]:
print('shape of pos_mean: ',pos_mean.shape,' Shape of neg_mean: ',neg_mean.shape) 

shape of pos_mean:  (6,)  Shape of neg_mean:  (6,)


In [25]:
pos_mean

sex                 1.437500
age                21.854167
Time                5.593750
Number_of_Warts     5.770833
Type                1.291667
Area               62.687500
dtype: float64

In [23]:
pos_std=np.mean(pos_feature, axis=0)
neg_std=np.mean(neg_feature, axis=0)

In [24]:
print('shape of pos_std: ',pos_std.shape,' Shape of neg_std: ',neg_std.shape) 

shape of pos_std:  (6,)  Shape of neg_std:  (6,)


In [27]:
neg_std

sex                 1.437500
age                21.854167
Time                5.593750
Number_of_Warts     5.770833
Type                1.291667
Area               62.687500
dtype: float64

In [36]:
# calculating covariance matrices
pos_cov=pos_feature.cov()
pos_cov=np.array(pos_cov).reshape(6,6)
neg_cov=neg_feature.cov()
neg_cov=np.array(neg_cov).reshape(6,6)

In [37]:
pos_cov

array([[ 2.51329787e-01, -3.17819149e-01, -1.80186170e-01,
         1.72872340e-02,  1.03723404e-01,  2.90558511e+00],
       [-3.17819149e-01,  5.73612589e+01,  4.18417553e+00,
        -1.92774823e+00,  3.20035461e-01,  6.25917553e+01],
       [-1.80186170e-01,  4.18417553e+00,  8.23038564e+00,
        -5.41888298e-01, -1.44946809e-01, -3.37965426e+00],
       [ 1.72872340e-02, -1.92774823e+00, -5.41888298e-01,
         1.38825355e+01, -1.68439716e-02,  1.95226064e+01],
       [ 1.03723404e-01,  3.20035461e-01, -1.44946809e-01,
        -1.68439716e-02,  3.81205674e-01,  2.63297872e-01],
       [ 2.90558511e+00,  6.25917553e+01, -3.37965426e+00,
         1.95226064e+01,  2.63297872e-01,  1.90268750e+03]])

In [88]:
#Fuction to calculate prior probability
def multivariate_prior(x,mean,cov):
    #we are going to build this function various steps, so that, it is easy to find error
    a=1/(((6.28)**len(pos_feature.T))*np.linalg.det(cov))**0.5 #1/under_root((2*pie)^k* determinant_of_covariance)
    b=(x-mean)
    c=np.linalg.inv(cov)
    d=np.dot(np.dot(b.T,c),b.T)
    return float(a*d)
    
def posterior(pos_prior,pos_prob,neg_prior,neg_prob):
    a= pos_prior*pos_prob
    b= neg_prior*neg_prob
    return a/(a+b),b/(a+b)

def accuracy(y_predicted,y_actual):
    #y_actual=y_actual.reshape(len(y_actual),1)
    return 100*(len(target[y_predicted==y_actual])/len(y_predicted))

In [89]:
pos_prob=len(pos_feature)/len(data)
neg_prob=1-pos_prob

In [90]:
predicted=[]
for i in range(len(feature)):
    pos , neg =posterior(multivariate_prior(feature[1],pos_mean,pos_cov),pos_prob,multivariate_prior(feature[1],neg_mean,neg_cov),neg_prob)
    if(pos>=neg):
        predicted.append(1)
    else:
        predicted.append(0)
        
print(accuracy(predicted,target))        

53.333333333333336


In [91]:
#just to check our accuracy
count=0
for i in range(len(data)):
    if(target.iloc[i]==predicted[i]):
        count=count+1
        
print(count/90) 

0.5333333333333333
